In [2]:
%load_ext autoreload
%autoreload 2


## classic pydata stack
import os 
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline 

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15,7)


In [15]:
import vanilla_NN as NN

In [16]:
num_blocks=10
dataset = NN.PolymerDataset(data_paths=["AA66466AA.npy","AA66266AA.npy"],num_blocks=num_blocks,nn=False,lstm=False)

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = dataset.data, dataset.labels
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=89)
classifier = RandomForestClassifier(random_state=89,n_estimators = 300 )
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {100*accuracy:.2f}%")

Accuracy: 84.23%


In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = dataset.data, dataset.labels
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=89)
classifier = GradientBoostingClassifier(random_state=89,n_estimators = 200)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {100*accuracy:.2f}%")

Accuracy: 93.93%


In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = dataset.data, dataset.labels
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=89)
classifier = GradientBoostingClassifier(random_state=89,n_estimators = 300)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {100*accuracy:.2f}%")

Accuracy: 94.18%


In [7]:
len(X_train[0])

90

In [8]:
np.array(dataset.data)
dataset.labels

array([0, 0, 0, ..., 1, 1, 1])

In [9]:
data_paths=["AA00400AA.npy","AA66466AA.npy"]
raw_data = [np.load(data_path, allow_pickle=True) for data_path in data_paths]

In [10]:
x[0]

NameError: name 'x' is not defined

In [ ]:
y[0]

NameError: name 'y' is not defined

In [ ]:
class PaddedDataset():

    def __init__(self, data_paths,seed=10):
        self.process(data_paths,seed)


    def process(self,data_paths,seed):

        raw_data = [np.load(data_path, allow_pickle=True) for data_path in data_paths]
        labels = [0,1]

    ## balance the dataset by removing signals that are too short or too long
    ## first we build the dataframe to know the lengths of the time series

        len_series = []

        for data in raw_data:
            lengths= []
            for row in data:
                ## length of time series
                lengths.append(row.shape[0])

            len_series.append(pd.Series(lengths))

        ## enforces that the first dataset is the smaller one in total size
        ## such that we can apply our balancing operations generally
        if len(len_series[0]) > len(len_series[1]):
            len_series.reverse()
            raw_data.reverse()
            labels.reverse()

        ## filter the dataset and remove signals that are:
        ## too short i.e. < len_series[0].quantile(0.1)
        ## too long i.e. > len_series[0].quantile(0.9)
        for i in range(2):
            mask = (len_series[i] > max(len_series[0].quantile(0.1),num_blocks)) & (len_series[i] < len_series[0].quantile(0.5))
            raw_data[i] = raw_data[i][mask]

        ## most likely, one dataset is still bigger than the other one
        ## therefore, we randomly sample data from the bigger dataset to create a new dataset of the same size as the small one 
        np.random.seed(seed=seed)

        # making sure the smallest dataset is the first one
        if len(raw_data[0]) > len(raw_data[1]):
            raw_data.reverse()
            labels.reverse()

        # randomly sampling and making a balanced dataset
        raw_data[1]  = np.random.permutation(raw_data[1])[:len(raw_data[0])]
        data=[]
        data_labels=[]
        
        max_length = max(len_series[0].max(), len_series[1].max())
        print("Max length",max_length)

        ## using our Input class to build the entire dataset and extracting features from each row
        for index, raw_data in enumerate(raw_data):
            for raw_series in raw_data:
                padded_series = self.pad(raw_series,max_length)
                data.append(padded_series)
                data_labels.append(labels[index])
        data = np.array(data)

        #normalizing features
        data = (data - data.mean(axis=0)) / data.std(axis=0)
        data_labels = np.array(data_labels)

        self.data = data
        self.labels = data_labels

        return self

    
    def pad(self,raw_series, max_length):

        l = max_length - raw_series.shape[0]
        x=raw_series[:,0]
        y= raw_series[:,1]

        if l > 0:
            res1 = np.concatenate((x, np.linspace(start= x[-1] + 0.01, stop= x[-1] + l*0.01 , num=l)),axis=None)
            res2 = np.concatenate((y, np.zeros(l)),axis=None)

        res = np.vstack((res1,res2)).T

        return res
        

In [ ]:
dataset = PaddedDataset(data_paths=["AA00400AA.npy","AA66466AA.npy"])

MemoryError: 

In [ ]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = dataset.data, dataset.labels
X_train, X_test, y_train, y_test = train_test_split(X, y)
classifier = TimeSeriesForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

ValueError: If passed as a np.array, X must be a 3-dimensional array, but found shape: (21243, 27)

In [ ]:
dataset.data[1]

array([ 0.17944997, -0.20482437,  0.67186573, -0.00658525,  0.94489608,
       -0.81025791, -0.73619943,  1.31484999, -0.43366814, -0.71692991,
       -0.57235341, -0.45231736, -0.82291628, -0.43769871, -0.81025791,
       -0.89015383,  1.00854561, -1.42100356, -0.48263948, -1.11383077,
        0.41667791, -0.81664598,  0.96985427, -0.81025791, -0.71873826,
        1.37042307, -0.68925605])